In [59]:
%cd /home/ead/iking5/code/graph_edit_rl

/home/ead/iking5/code/graph_edit_rl


/home/ead/iking5/miniconda3/envs/edit_dist/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning:

This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.



In [60]:
import torch
from models.action_encoder import HVAE
args,kwargs,sd = torch.load('action_enc.pt')
embedder = HVAE(*args, **kwargs)
embedder.load_state_dict(sd)

/tmp/ipykernel_4143197/2746230117.py:3: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



<All keys matched successfully>

In [61]:
from environment.actions import N_ACTIONS, IDX_TO_ACTION, SIMPLE_ACTIONS, N1, N2, N3, N4
from generator import N_COLORS, encode_actions



categories, colors, names, actions, params = [],[],[],[],[]
for i in range(N_ACTIONS):
    for j in range(N_COLORS):
        actions.append(i)
        params.append(j)
        names.append(
            f'{IDX_TO_ACTION[i].__name__}, {j}'
        )

        a = IDX_TO_ACTION[i]
        if a in SIMPLE_ACTIONS:
            categories.append('S')
        elif a in N1:
            categories.append('N1')
        elif a in N2:
            categories.append('N2')
        elif a in N3:
            categories.append('N3')
        else:
            categories.append('N4')

        colors.append(j)

action_vecs = encode_actions(torch.tensor(actions), torch.tensor(params))

In [62]:
embedder.eval()
with torch.no_grad():
    action_z = embedder.encode_action(action_vecs)[0]

In [64]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=)
z2d = tsne.fit_transform(action_z)

df = pd.DataFrame(
    dict(
        x=z2d[:,0], y=z2d[:,1],
        categories=categories,
        name=names,
        ntypes=colors
    )
)

fig = px.scatter(df, x='x', y='y', hover_data=['name'], color='categories')
fig.show()

ValueError: perplexity must be less than n_samples